# Choosing a neuron model

In [ ]:
import sinabs
import sinabs.layers as sl
import torch
import matplotlib.pyplot as plt

Let's start by creating a helpful plotting function and some constant current input.

In [ ]:
def plot_evolution(neuron_model: sinabs.layers, input: torch.Tensor):
    neuron_model.reset_states()
    output = neuron_model(input)

    plt.figure(figsize=(10, 3))
    for key, recording in neuron_model.recordings.items():
        plt.plot(recording.flatten(), drawstyle="steps", label=key)
    plt.plot(
        output.detach().flatten(), drawstyle="steps", color="black", label="output"
    )
    if not "spike_threshold" in neuron_model.recordings:
        plt.plot(
            [neuron_model.spike_threshold] * input.shape[1], label="spike_threshold"
        )
    plt.xlabel("time")
    plt.title(f"{neuron_model.__class__.__name__} neuron dynamics")
    plt.legend()


const_current = torch.ones((1, 100, 1)) * 0.03
single_current = torch.zeros((1, 100, 1))
single_current[:, 0] = 0.1

## Integrate and Fire neuron
This neuron has no leakage and simply integrates all the input it receives. It emits a spike whenever the membrane potential is above the spike threshold.

In [ ]:
iaf_neuron = sl.IAF(record_states=True)
plot_evolution(iaf_neuron, const_current)

We can activate synaptic currents in this neuron model by setting `tau_syn`. All inputs will be integrated to its i_syn state, which will then be decayed and added to the membrane potential at every step. In the following plot we only provide an input at the first time step.

In [ ]:
iaf_neuron = sl.IAF(tau_syn=15.0, record_states=True)
plot_evolution(iaf_neuron, single_current)

## Leaky Integrate and Fire neuron
This neuron integrates the input and decays its state at every time step.

In [ ]:
lif_neuron = sl.LIF(tau_mem=40.0, norm_input=False, record_states=True)
plot_evolution(lif_neuron, const_current)

By default, no synaptic dynamics are used. We can enable that by setting tau_syn. Note that instead of a constant current, we now provide input only at the first time step. 

In [ ]:
lif_neuron = sl.LIF(tau_mem=40.0, tau_syn=30.0, norm_input=False, record_states=True)
plot_evolution(lif_neuron, single_current)

## Leaky Integrator neuron
Same as LIF, just without activation function.

In [ ]:
exp_leak_neuron = sl.ExpLeak(tau_mem=60.0, record_states=True)
plot_evolution(exp_leak_neuron, const_current)

## Adaptive Leaky Integrate and Fire neuron
This is a LIF neuron with an adaptive threshold.

In [ ]:
alif_neuron = sl.ALIF(
    tau_mem=40.0, tau_adapt=40.0, adapt_scale=20, norm_input=False, record_states=True
)
plot_evolution(alif_neuron, const_current * 4)